In [47]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from tqdm import tqdm
import torch

import sys
sys.path.append('../')

from redkg.dataloader import TrainDataset, get_info
from redkg.models.kge import KGEModel
from redkg.config import Config
from redkg.utils import AttributeDict
from redkg.train import train_kge_model

from ogb.linkproppred import LinkPropPredDataset, Evaluator


def test_model(kge_model, args):
    metrics = defaultdict()
    metrics_temp = kge_model.test_step(kge_model, test_triples, args, random_sampling=False)
    for metric in metrics_temp:
        metrics[metric] = metrics_temp[metric]
            
    return metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Загрузка данных
## BioKG
<img src="biokg_logo2.png" width="400">

Набор данных **biokg** представляет собой граф знаний (ГЗ), который был создан с использованием данных из большого количества хранилищ биомедицинских данных. Он содержит 5 типов сущностей: болезни (10 687 узлов), белки (17 499 узлов), лекарства (10 533 узла), побочные эффекты (9 969 узлов) и функции белков (45 085 узлов). Существует 51 тип направленных отношений, связывающих два типа сущностей, в том числе 38 видов взаимодействий лекарство-лекарство, 8 видов взаимодействия белок-белок, а также отношения лекарство-белок, лекарство-побочное действие, функция-функция. Все отношения моделируются как направленные ребра, среди которых отношения, соединяющие одни и те же типы сущностей (например, белок-белок, лекарство-лекарство, функция-функция), всегда симметричны, т. е. ребра двунаправлены.

Этот набор данных имеет отношение как к биомедицинским, так и к фундаментальным исследованиям машинного обучения. С точки зрения биомедицины набор данных позволяет нам лучше понять биологию человека и генерировать прогнозы, которые могут направлять дальнейшие биомедицинские исследования. С фундаментальной стороны ML набор данных создает проблемы при обработке зашумленного, неполного ГЗ с возможными противоречивыми наблюдениями. Это связано с тем, что набор данных ogbl-biokg включает в себя гетерогенные взаимодействия, которые охватывают диапазон от молекулярного масштаба (например, межбелковые взаимодействия внутри клетки) до целых популяций (например, сообщения о нежелательных побочных эффектах, испытываемых пациентами в конкретной стране). Кроме того, триплеты в KG поступают из источников с различными уровнями достоверности, включая экспериментальные показания, аннотации, созданные человеком, и автоматически извлеченные метаданные.

In [50]:
dataset_name = 'ogbl-biokg'

dataset = LinkPropPredDataset(name = dataset_name, root = '../data')
split_edge = dataset.get_edge_split()
train_triples, valid_triples, test_triples = split_edge["train"], split_edge["valid"], split_edge["test"]

info = get_info(dataset, train_triples, do_count = True)

# Постановка задачи
Решается задача прогнозирования связей в ГЗ используя модель KGE из модуля KGEModel



In [51]:
evaluator = Evaluator(name = dataset_name)

kge_model = KGEModel(
        model_name="TransE",
        nentity=info['nentity'],
        nrelation=info['nrelation'],
        hidden_dim=128,
        gamma=12.0,
        double_entity_embedding=True,
        double_relation_embedding=True,
        evaluator=evaluator
    )

## Обучение модели

In [26]:
train_pars = AttributeDict()
train_pars.cuda = False
train_pars.uni_weight = True
train_pars.negative_adversarial_sampling = True
train_pars.regularization = 0.0
train_pars.adversarial_temperature = 1.0
train_pars.train_batch_size = 128
train_pars.negative_sample_size = 128
train_pars.learning_rate = 0.001
train_pars.cpu_num = 10
train_pars.negative_mode = "full"
train_pars.neg_size_eval_train = 500
train_pars.test_log_steps = 1000
train_pars.test_batch_size = 128
train_pars.nentity = info['nentity']  
train_pars.nrelation = info['nrelation']
train_pars.do_test = True

## Результаты

In [12]:
evaluator = Evaluator() 

kge_model = KGEModel(
        model_name=TransE,
        nentity=info['nentity'],
        nrelation=info['nrelation'],
        hidden_dim=128,
        gamma=12.0,
        double_entity_embedding=True,
        double_relation_embedding=True,
        evaluator=evaluator
    )

training_logs, test_logs = train_kge_model(kge_model, train_pars, info, train_triples, valid_triples)
metrics = test_model(kge_model, args)

for key,value in metrics.items():
    print(f'{key}: {round(value,3)}')

100%|██████████| 100/100 [00:05<00:00, 16.91it/s]


mrr: 0.721
hits1: 0.438
hits3: 0.715
hits10: 0.965
